# DASC5301 Data Science, Fall 2021, Chengkai Li, Unversity of Texas at Arlington
# Programming Assignment 1 Solution

In [ ]:
import requests
page = requests.get("https://www.nps.gov/robots.txt")

In [ ]:
import requests
import re
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from bs4 import BeautifulSoup
from collections import defaultdict

In [ ]:
#Retrieve the first page.
page = requests.get("https://www.nps.gov/index.htm")
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
#Find the list of all states.
text_loc = soup.find("a", text="Alabama")
text_loc = text_loc.parent
all_states_li = text_loc.findNextSiblings("li")

all_states_a = []
tmp_a = text_loc.findChild('a')
all_states_a.append(tmp_a)

for li in all_states_li: 
    tmp_a = li.findChild('a')
    all_states_a.append(tmp_a)

In [ ]:
#Extract each park's contact info. 
def park_info(park_link):
    page = requests.get(park_link)
    soup = BeautifulSoup(page.content, 'html.parser')

    text_loc = soup.find('h4', text="Mailing Address:")
    if text_loc == None: 
        address = " "
    else:
        address_div = text_loc.findNextSibling('div')
        if address_div == None: 
            address_p = text_loc.findNextSibling('p')
            address = " " + address_p.text
        else: 
            address_p = address_div.findChild('p')
            address_spans = address_p.findChildren('span')
            address = " " + address_spans[0].text + address_spans[1].text

    text_loc = soup.find("h4", text="Phone:")
    if text_loc == None: 
        park_phone = " "
    else: 
        phone_p = text_loc.findNextSibling('p')
        phone_span = phone_p.findChild('span')
        park_phone = phone_span.text
    
    #Address
    addressLine1 = address
    addressLine2 = " "
    addressLine3 = " "
    city = " "
    state = " "
    zipcode = " "

    if address != '' and  not address.isspace(): 
      addressLines = address.split('\n')
      if addressLines[-1] == ' ' or addressLines[-1] == '': 
        addressLines = addressLines[:-1]
      if addressLines[0] == ' ' or addressLines[0] == '':
        addressLines = addressLines[1:]
      
      if len(addressLines) >= 3: 
        zipcode = addressLines[-1]
        city_state = addressLines[-2].split(',')
        city = city_state[0]
        state = city_state[1]
        addressLine1 = addressLines[0]
      if len(addressLines) >= 4: 
        addressLine2 = addressLines[1]
      if len(addressLines) >= 5: 
        addressLine3 = addressLines[2]

    return addressLine1, addressLine2, addressLine3, city, state, zipcode, park_phone 

In [ ]:
parks_list = []
#Retrieve all states pages.
for a in all_states_a: 
    state = a.text
    href = a['href']
    state_link = "https://www.nps.gov" + href
    page = requests.get(state_link)
    soup = BeautifulSoup(page.content, 'html.parser')

    text_loc = soup.find('ul', id="list_parks")
    all_h2s = text_loc.findAllNext('h2')
    #Retrieve parks pages. 
    for h2 in all_h2s:
        park_category = h2.text
        h3_name = h2.findNext('h3')
        if h3_name == None:
            continue
        a_name = h3_name.findChild('a')
        if a_name == None: 
            continue
        park_link = "http://www.nps.gov" + a_name['href'] + "index.htm"
        if park_link == "http://www.nps.gov/es-es/mora/index.htm": 
            continue
        park_name = a_name.text
        p_desc = h2.findNext('p')
        park_desc = p_desc.text
      
        addressLine1, addressLine2, addressLine3, city, state, zipcode, park_phone = park_info(park_link)
        parks_list.append([park_name, state, park_category, park_desc, addressLine1, addressLine2, addressLine3, city, state, zipcode, park_phone])

In [ ]:
#Saving the data into a csv file.
df = pd.DataFrame(parks_list, columns = ['name', 'state', 'category','Description', 'addressLine1', 'addressLine2', 'addressLine3', 'city', 'state', 'zipcode', 'phone'])
df.to_csv('parks_data.csv')